# TWO BODY PROBLEM NEWTONIAN

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def rel_len(x_1, x_2):
    return np.sqrt((x_1[0] - x_2[0])**2 + (x_1[1] - x_2[1])**2)

In [2]:
def acceleration(x_1, x_2):
    r = rel_len(x_1, x_2)
    a_1 = - np.subtract(x_1, x_2)/(2*r**3)
    a_2 = - np.subtract(x_2, x_1)/(2*r**3)
    return np.array([a_1, a_2])

def boost(v_1i, v_2i, a_1, a_2, dt):
    v_1 = np.add(v_1i, a_1*dt)
    v_2 = np.add(v_2i, a_2*dt)
    return np.array([v_1, v_2])

def move(x_1i, x_2i, v_1, v_2, dt):
    x_1 = np.add(x_1i, v_1*dt)
    x_2 = np.add(x_2i, v_2*dt)

    return np.array([x_1, x_2])

In [3]:
def twoOrbsNewt(s, t_max = 2000000, d=5, dt = 1000):
    x_1, x_2, v_1, v_2, a_1, a_2 = s

    dist_max = d*rel_len(x_1, x_2)
    
    pos1 = [], []
    pos2 = [], []
    vel1 = [], []
    vel2 = [], []
    acc1 = [], []
    acc2 = [], []

    for i in np.arange(0,t_max,dt):
        pos1[0].append(x_1[0])
        pos1[1].append(x_1[1])
        pos2[0].append(x_2[0])
        pos2[1].append(x_2[1])
        vel1[0].append(v_1[0])
        vel1[1].append(v_1[1])
        vel2[0].append(v_2[0])
        vel2[1].append(v_2[1])
        acc1[0].append(a_1[0])
        acc1[1].append(a_1[1])
        acc2[0].append(a_2[0])
        acc2[1].append(a_2[1])

        # dist = rel_len(x_1, x_2)
        # if dist >= dist_max:
        #     break
        a_1, a_2 = acceleration(x_1, x_2)
        v_1, v_2 = boost(v_1, v_2, a_1, a_2, dt)
        x_1, x_2 = move(x_1, x_2, v_1, v_2, dt)

    return pos1, pos2, vel1, vel2, acc1, acc2

# TWO BODY PROBLEM EIH

In [4]:
def rel_len(x_1, x_2):
    return np.sqrt((x_1[0]-x_2[0])**2 + (x_1[1]-x_2[1])**2)

In [5]:
#Takes numpy arrays as input!!

def accelerationEIH(r_1, r_2, v_1, v_2, a_1, a_2, m_1, m_2):
    r_vec = r_1 - r_2
    v_vec = v_1 - v_2
    r = rel_len(r_1,r_2)
    v = rel_len(v_1,v_2)

    acc_1 = np.array([0,0])
    acc_2 = np.array([0,0])
    for j in range(2):
        acc_1[j] = 1/(1 + 3/2*v_1[j]**2 + 3*m_2/r)*(m_2/(2*r)*(-6*v_1[j]*v/r + 7*v_2[j]*v/r + 7*a_2[j]
            + np.dot(a_2, r_vec)*r_vec[j]/r**2 + np.dot(v_2, v_vec)*r_vec[j]/r**2 - v_2[j]*np.dot(v_2, r_vec)/r**2
            - np.dot(v_2, r_vec)*r_vec[j]*v/r**3 + 2*r_vec[j]/r**2 - 3*np.dot(v_1,v_1)*r_vec[j]/r**2
            - 3*np.dot(v_2,v_2)*r_vec[j]/r**2 + 7*np.dot(v_1,v_2)*r_vec[j]/r**2 - np.dot(v_2, r_vec)*np.dot(v_1, r_vec)*r_vec[j]/r**4)
            - v_2[j]*np.dot(v_1, r_vec)/r**2 - 2*(m_1 + m_2)* r_vec[j]/r**3)

        acc_2[j] = 1/(1 + 3/2*v_2[j]**2 + 3*m_1/r)*(m_1/(2*r)*(-6*v_2[j]*v/r + 7*v_1[j]*v/r + 7*a_1[j]
            + np.dot(a_1, r_vec)*r_vec[j]/r**2 + np.dot(v_1, v_vec)*r_vec[j]/r**2 + v_1[j]*np.dot(v_1, r_vec)/r**2
            - np.dot(v_1, r_vec)*r_vec[j]*v/r**3 - 2*r_vec[j]/r**2 + 3*np.dot(v_1,v_1)*r_vec[j]/r**2
            + 3*np.dot(v_2,v_2)*r_vec[j]/r**2 - 7*np.dot(v_1,v_2)*r_vec[j]/r**2 - v_1[j]*np.dot(v_2, r_vec)/r**2 
            + np.dot(v_2, r_vec)*np.dot(v_1, r_vec)*r_vec[j]/r**4) + 2*(m_1 + m_2)*r_vec[j]/r**3)

    return acc_1, acc_2

def boostEIH(v_1, v_2, a_1, a_2, dt):
    v_1 = v_1 + a_1*dt
    v_2 = v_2 + a_2*dt
    return v_1, v_2

def moveEIH(r_1, r_2, v_1, v_2, dt):
    r_1 = r_1 + v_1*dt
    r_2 = r_2 + v_2*dt
    return r_1, r_2


In [6]:
r_1 = np.array([123,-97.4])
r_2 = np.array([-291,223])
v_1 = np.array([-1,1])
v_2 = np.array([2,-2])
a_1 = np.array([1,2])
a_2 = np.array([3,2])
m_1 = 3
m_2 = 5


dt = 10
t = np.arange(0,1000,dt)

pos1 = [], []

for i in t:
    pos1[0].append(r_1[0])
    pos1[1].append(r_1[1])
    # pos2.append(r_2)
    # vel1.append(v_1)
    # vel2.append(v_2)
    # acc1.append(a_1)
    # acc2.append(a_2)
    a_1, a_2 = accelerationEIH(r_1, r_2, v_1, v_2, a_1, a_2, m_1, m_2)
    v_1, v_2 = boostEIH(v_1, v_2, a_1, a_2, dt)
    r_1, r_2 = moveEIH(r_1, r_2, v_1, v_2, dt)
    


# plt.plot(pos1[0], pos1[1])


In [7]:
len(pos1)

2